# COVID-19 por comuna
> Nuevos casos confirmados por comuna.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import pandas as pd
import altair as alt

In [2]:
#hide
county_cumulative_cases = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv")

In [3]:
#hide
county_cumulative_cases_RM = county_cumulative_cases.query("Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Poblacion", "Tasa", "Codigo comuna"])

In [4]:
#hide
county_cases_RM = county_cumulative_cases_RM.set_index("Comuna").T.diff().dropna()

In [5]:
#hide
county_cases_RM["Total"] = county_cases_RM.sum(axis=1)
county_cases_RM = county_cases_RM.T.apply(lambda x: x/x[-1]).T

In [6]:
#hide
start_date = county_cases_RM.reset_index()["index"].iloc[0]
end_date = county_cases_RM.reset_index()["index"].iloc[-1]

data = pd.DataFrame()
data['Fecha'] = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')
data = data.set_index("Fecha")

In [7]:
#hide
data[list(county_cases_RM.columns)] = county_cases_RM[list(county_cases_RM.columns)]

In [8]:
#hide
data = data.interpolate(method="linear")

In [9]:
#hide
regional_cumulative_cases = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv')

In [10]:
#hide
regional_cumulative_cases_RM = regional_cumulative_cases.query("Region=='Metropolitana'").iloc[-1].drop("Region")
regional_cases_RM = regional_cumulative_cases_RM.diff().dropna()

In [11]:
#hide
data["Regional"] = regional_cases_RM

In [12]:
#hide
data = data.T.apply(lambda x: x[:-1]*x[-1]).T.astype(int)

In [13]:
#hide
data_long_form = data.drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Casos confirmados")

In [14]:
#hide_input
input_dropdown = alt.binding_select(options=data_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='category20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(data_long_form).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="nuevos casos confirmados")),
    tooltip = ["Fecha", "Comuna", "Casos confirmados"],
    color = color
).properties(
    title="Nuevos casos confirmados por comuna en la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

In [15]:
#hide
list_nororiente = ['La Reina', 'Las Condes', 'Lo Barnechea', 'Nunoa',  'Providencia', 'Vitacura']
list_suroriente = ["Macul", "La Florida", 'Penalolen', "Puente Alto"]
list_norponiente = ['Cerro Navia', 'Lo Prado', 'Pudahuel', 'Quinta Normal', 'Renca']
list_surponiente = ['Estacion Central', 'Maipu', 'Cerrillos', 'Padre Hurtado', 'Penaflor']
list_norte = ['Conchali', 'Huechuraba', 'Independencia', 'Recoleta', 'Quilicura']
list_centro = ['Santiago']
list_sur = ['Pedro Aguirre Cerda', 'San Miguel', 'San Joaquin', 'Lo Espejo', 'La Cisterna', \
           'La Granja', 'San Ramon', 'El Bosque', 'San Bernardo', 'La Pintana']

In [16]:
#hide
cases_nororiente = data[list_nororiente]
cases_suroriente = data[list_suroriente]
cases_norponiente = data[list_norponiente]
cases_surponiente = data[list_surponiente]
cases_norte = data[list_norte]
cases_centro = data[list_centro]
cases_sur = data[list_sur]

In [17]:
#hide
data = pd.DataFrame()
data["7. Nororiente"] = cases_nororiente.sum(axis=1)
data["6. Suroriente"] = cases_suroriente.sum(axis=1)
data["5. Norte"] = cases_norte.sum(axis=1)
data["4. Centro"] = cases_centro.sum(axis=1)
data["3. Sur"] = cases_sur.sum(axis=1)
data["2. Norponiente"] = cases_norponiente.sum(axis=1)
data["1. Surponiente"] = cases_surponiente.sum(axis=1)

In [18]:
data = data.reset_index()

In [19]:
#hide
data = data.melt("Fecha", var_name="Sector", value_name="Casos confirmados")

In [20]:
#hide_input
chart1 = alt.Chart(data).mark_area().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False)),
    y=alt.Y("Casos confirmados", axis=None, stack="center"),
    tooltip = ["Fecha", "Casos confirmados"],
    color = alt.Color("Sector"),
).properties(
    title="Nuevos casos confirmados en la R.M.",
    width=700)

chart2 = alt.Chart(data).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, ticks=False, labels=False)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="", format='%'), stack="normalize"),
    tooltip = ["Fecha", "Casos confirmados"],
    color = alt.Color("Sector"),
).properties(
    height = 100,
    width=700
)

alt.vconcat(
    chart1,
    chart2
).configure_concat(
    spacing=0
)

alt.VConcatChart(...)